In [297]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import scipy.sparse as sparse

### Creating Training Data

In [459]:
tempfile = open('Variables/trainImagePatchArray.pckl', 'rb')
trainImagePatchArray = pickle.load(tempfile)
tempfile.close()
trainingDataX= trainImagePatchArray.astype(int)

In [421]:
tempfile = open('Variables/trainingOutputImage.pckl', 'rb')
trainingOutputImage = pickle.load(tempfile)
tempfile.close()
trainingOutputImage = trainingOutputImage[1:300,1:224]
trainout = trainingOutputImage.reshape(((trainingOutputImage.shape[0]*trainingOutputImage.shape[1]),3))

In [422]:
tempfile = open('Variables/clusters.pckl', 'rb')
clusters = pickle.load(tempfile)
tempfile.close()
colorlist = list(clusters.keys())
colors = np.asarray(colorlist,dtype=int)

In [423]:
Trainingdataframe = pd.DataFrame(data=trainingDataX,index=range(1,len(trainingDataX)+1), columns= ["px"+str(i) for i in range(1,10)] )
arr = sparse.coo_matrix(trainout)
Trainingdataframe['Color'] = arr.toarray().tolist()
Trainingdataframe

,px1,px2,px3,px4,px5,px6,px7,px8,px9,Color
1,88,99,90,110,114,93,139,107,95,"[78, 102, 59]"
2,99,90,92,114,93,85,107,95,99,"[78, 102, 59]"
3,90,92,82,93,85,85,95,99,91,"[78, 102, 59]"
4,92,82,80,85,85,93,99,91,114,"[78, 102, 59]"
5,82,80,91,85,93,100,91,114,118,"[78, 102, 59]"
...,...,...,...,...,...,...,...,...,...,...
66673,203,203,203,204,204,203,204,204,204,"[213, 203, 153]"
66674,203,203,203,204,203,203,204,204,203,"[213, 203, 153]"
66675,203,203,202,203,203,202,204,203,202,"[213, 203, 153]"
66676,203,202,201,203,202,201,203,202,201,"[213, 203, 153]"


In [424]:
trainingDataX

array([[ 88,  99,  90, ..., 139, 107,  95],
       [ 99,  90,  92, ..., 107,  95,  99],
       [ 90,  92,  82, ...,  95,  99,  91],
       ...,
       [203, 203, 202, ..., 204, 203, 202],
       [203, 202, 201, ..., 203, 202, 201],
       [202, 201, 201, ..., 202, 201, 204]])

In [425]:
trainingDataY = np.zeros((len(trainout),5),dtype=int)
for index,pixel in enumerate(trainout):
    trainingDataY[index,np.where((colors == pixel).all(axis=1))] = 1

In [153]:
trainingDataY

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

### Creating Testing Data. 

In [426]:
tempfile = open('Variables/testImagePatchArray.pckl', 'rb')
testImagePatchArray = pickle.load(tempfile)
tempfile.close()
testDataX = testImagePatchArray.astype(int)
tempfile = open('Variables/expectedtestingOutputImage.pckl', 'rb')
testOutputImage = pickle.load(tempfile)
tempfile.close()
testOutputImage = testOutputImage[1:300,1:224]
testout = testOutputImage.reshape(((testOutputImage.shape[0]*testOutputImage.shape[1]),3))

In [427]:
Testingdataframe = pd.DataFrame(data=testDataX,index=range(1,len(testDataX)+1), columns= ["px"+str(i) for i in range(1,10)] )
arr = sparse.coo_matrix(testout)
Testingdataframe['Color'] = arr.toarray().tolist()
Testingdataframe

,px1,px2,px3,px4,px5,px6,px7,px8,px9,Color
1,54,55,56,53,54,55,51,52,53,"[36, 82, 189]"
2,55,56,57,54,55,56,52,53,55,"[36, 82, 189]"
3,56,57,57,55,56,57,53,55,56,"[36, 82, 189]"
4,57,57,57,56,57,57,55,56,56,"[36, 82, 189]"
5,57,57,57,57,57,57,56,56,56,"[36, 82, 189]"
...,...,...,...,...,...,...,...,...,...,...
66673,123,129,113,108,108,99,103,97,97,"[78, 102, 59]"
66674,129,113,110,108,99,96,97,97,93,"[78, 102, 59]"
66675,113,110,97,99,96,94,97,93,101,"[78, 102, 59]"
66676,110,97,97,96,94,98,93,101,108,"[78, 102, 59]"


In [170]:
testDataY = np.zeros((len(testout),5),dtype=int)
for index,pixel in enumerate(testout):
    testDataY[index,np.where((colors == pixel).all(axis=1))] = 1

In [428]:
testDataX

array([[ 54,  55,  56, ...,  51,  52,  53],
       [ 55,  56,  57, ...,  52,  53,  55],
       [ 56,  57,  57, ...,  53,  55,  56],
       ...,
       [113, 110,  97, ...,  97,  93, 101],
       [110,  97,  97, ...,  93, 101, 108],
       [ 97,  97,  94, ..., 101, 108, 114]])

In [429]:
testDataY

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0]])

### Shuffling data

In [461]:
trainingDataXnorm = (trainingDataX - np.mean(trainingDataX,axis=0))/np.std(trainingDataX,axis=0)

In [462]:
shuffledindex = np.random.choice(range(0,len(trainingDataXnorm)),len(trainingDataXnorm),replace=False)

In [463]:
trainingDataXnorm = trainingDataXnorm[shuffledindex]
trainingDataY = trainingDataY[shuffledindex]

In [504]:
trainingDataY

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0]])

In [495]:
#softmax function
def softmax(XW):
    XW -= np.max(XW)
    prob = (np.exp(XW).T/ np.sum(np.exp(XW),axis=1)).T
    return prob

#loss function
def loss(W,Y,prob):
    N = len(Y)
    lvalue = ((-1 / N) * np.sum(Y * np.log(prob)))
    return lvalue
    
#loss function
def gradient(W,X,Y,prob):
    N = len(Y)
    galue = ((-1 / N) * np.dot(X.T,(Y - prob)))
    return galue
    
#loss function
def predict(prob):
    preds = np.argmax(prob,axis=1)
    predcolors = [colors[index] for index in preds]
    return np.asmatrix(predcolors)

#accuracy
def accuracy(predicted,actual):
    accuracy = (np.sum(np.equal(predicted,actual).all(axis=1))/len(actual))*100
    return accuracy
    
#return minibatch
def minibatch(X,Y,batchSize):
    initialpoint = np.random.randint(0, Y.shape[0] - batchSize - 1) 
    Xbatch = X[initialpoint:(initialpoint + batchSize)] 
    Ybatch = Y[initialpoint:(initialpoint + batchSize)]
    return Xbatch,Ybatch
    

In [515]:
W = np.zeros((9,5))
batchSize = 1000
alpha = 1
previousloss = 0
minibatchX,minibatchY = minibatch(trainingDataXnorm,trainingDataY,batchSize)   
for i in range(0,100): 
    rawscore = np.dot(minibatchX,W)
    prob = softmax(rawscore)
    gred = gradient(W,minibatchX,minibatchY,prob)
    W = W - (alpha*gred)
    loss_value = loss(W,minibatchY,prob)
    print(loss_value)
    previousloss = loss_value
    np.no
print(i)
print(W)

    

1.6094379124341003
1.6070736097447385
1.6058243837036652
1.6050947993583529
1.6045708934493692
1.6041459180055868
1.603770907456028
1.6034261425390155
1.6031025893479578
1.602796003223863
1.6025041088031418
1.602225500266529
1.6019591706768406
1.6017043069638557
1.6014602064680226
1.601226235941866
1.6010018148299363
1.6007864050551015
1.6005795061411032
1.6003806512080911
1.6001894043684999
1.6000053582383105
1.599828131933535
1.5996573691325189
1.5994927363724682
1.5993339214284557
1.599180631841088
1.599032593532504
1.598889549533412
1.5987512587943737
1.5986174950868737
1.598488045980447
1.5983627118950121
1.5982413052201419
1.598123649498286
1.5980095786661928
1.5978989363510587
1.5977915752169742
1.59768735635832
1.5975861487365024
1.5974878286569942
1.5973922792836652
1.597299390187723
1.5972090569287067
1.5971211806652041
1.5970356677931101
1.5969524296094146
1.5968713819996463
1.5967924451472504
1.5967155432632871
1.596640604334975
1.5965675598916986
1.5964963447872087
1.59642